# Othello

Othello is a turn-based two-player strategy board game. The players take turns placing pieces, one player white and the other player black, on an 8x8 board in such a way that captures some of the opponent's pieces, with the goal of finishing the game with more pieces of their color on the board.

Every move must capture one or more of the opponent's pieces. To capture, player A places a piece adjacent to one of player B's pieces so that there is a straight line (horizontal, vertical or diagonal) of adjacent pieces that begins with one of player A's pieces, continues with one or more of player B's pieces, and ends with one of player A's pieces.

For example, if Black places a piece on square (5, 1), he will capture all of Black's pieces (5, 1) and (5, 6):

<img src='files/img/capture.png'>

For more information about the game (which is also known as Reversi) including detailed rules, see the entry on [Wikipedia](https://en.wikipedia.org/wiki/Reversi). Additionally, this implementation doesn't take into account some tournament-style Othello details, such as game time limits and a different indexing scheme.

We will implement representations for the board and pieces and the mechanics of playing a game. We will then explore several game-playing strategies. There is a simple command-line program provided for playing against the computer or comparing two strategies.

Written by [Daniel Connelly](http://dhconnelly.com). This implementation follows chapter 18 of Peter Norvig's "Paradigms of Artificial Intelligence".

## Table of contents

1. [Board representation](#board)
2. [Playing the game](#playing)
3. [Strategies](#strategies)
   - [Random](#random)<br>
   - [Local maximization](#localmax)<br>
   - [Minimax search](#minimax)<br>
   - [Alpha-beta search](#alphabeta)<br>
4. [Conclusion](#conclusion)

<a id="board"></a>

## Board Representation

We represent the board as a 100-element list, which includes each square on the board as well as the outside edge. Each consecutive sublist of ten elements represents a single row, and each list element stores a piece. An initial board contains four pieces in the center:

```
     ? ? ? ? ? ? ? ? ? ?
     ? . . . . . . . . ?
     ? . . . . . . . . ?
     ? . . . . . . . . ?
     ? . . . o @ . . . ?
     ? . . . @ o . . . ?
     ? . . . . . . . . ?
     ? . . . . . . . . ?
     ? . . . . . . . . ?
     ? ? ? ? ? ? ? ? ? ?
```

This representation has two useful properties:
1. Square (m, n) can be accessed as `board[mn]`. This avoids the need to write functions that convert between square locations and list indexes.
2. Operations involving bounds checking are slightly simpler.

The outside edge is marked `?`, empty squares are `.`, black is `@`, and white is `o`. The black and white pieces represent the two players.

In [1]:
EMPTY, BLACK, WHITE, OUTER = '.', '@', 'o', '?'
PIECES = (EMPTY, BLACK, WHITE, OUTER)
PLAYERS = {BLACK: 'Black', WHITE: 'White'}

To refer to neighbor squares we can add a direction to a square.

In [2]:
UP, DOWN, LEFT, RIGHT = -10, 10, -1, 1
UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, UP_LEFT = -9, 11, 9, -11
DIRECTIONS = (UP, UP_RIGHT, RIGHT, DOWN_RIGHT, DOWN, DOWN_LEFT, LEFT, UP_LEFT)

In [3]:
def squares():
    """List all the valid squares on the board"""
    return [i for i in range(11, 89) if 1 <= (i % 10) <= 8]

def initial_board():
    """Create a new board with the initial black and white positions filled"""
    board = [OUTER] * 100
    for i in squares():
        board[i] = EMPTY
    # The middle four squares should hold the initial piece positions.
    board[44], board[45] = WHITE, BLACK
    board[54], board[55] = BLACK, WHITE
    return board

def print_board(board):
    """Get a string representation of the board."""
    rep = ''
    rep += '  {0}\n'.format(' '.join(map(str, range(1, 9))))
    for row in range(1, 9):
        begin, end = 10*row + 1, 10*row + 9
        rep += '{0} {1}\n'.format(row, ' '.join(board[begin:end]))
    return rep

print(print_board(initial_board()))

  1 2 3 4 5 6 7 8
1 . . . . . . . .
2 . . . . . . . .
3 . . . . . . . .
4 . . . o @ . . .
5 . . . @ o . . .
6 . . . . . . . .
7 . . . . . . . .
8 . . . . . . . .



<a id="playing"></a>

## Playing the Game

We need functions to get moves from players, check to make sure that the moves are legal, apply the moves to the board, and detect when the game is over.

### Checking Moves

A move must be both valid and legal: it must refer to a real square, and it must form a bracket with another piece of the same color with pieces of the opposite color in between.

In [4]:
def is_valid(move):
    """Is move a square on the board?"""
    return isinstance(move, int) and move in squares()

def opponent(player):
    """Get player's opponent piece."""
    return BLACK if player is WHITE else WHITE

def find_bracket(square, player, board, direction):
    """
    Find a square that forms a bracket with `square` for `player` in the given
    `direction`. Returns None if no such squares exists.
    """
    bracket = square + direction
    if board[bracket] == player:
        return None
    opp = opponent(player)
    while board[bracket] == opp:
        bracket += direction
    return None if board[bracket] in (OUTER, EMPTY) else bracket

def is_legal(move, player, board):
    """Is this a legal move for the player?"""
    hasbracket = lambda direction: find_bracket(move, player, board, direction)
    return board[move] == EMPTY and any(map(hasbracket, DIRECTIONS))

### Making Moves

When the player makes a move, we need to update the board and flip all the bracketed pieces.

In [6]:
def make_move(move, player, board):
    """Update the board to reflect the move by the specified player."""
    board[move] = player
    for d in DIRECTIONS:
        make_flips(move, player, board, d)
    return board

def make_flips(move, player, board, direction):
    """Flip pieces in the given direction as a result of the move by player."""
    bracket = find_bracket(move, player, board, direction)
    if not bracket:
        return
    square = move + direction
    while square != bracket:
        board[square] = player
        square += direction